In [ ]:
len(ytest_encoding['input_ids'])

1590

In [ ]:
import tensorflow as tf
import numpy
train_dataset=tf.data.Dataset.from_tensor_slices((dict(xtrain_encoding),ytrain_encoding['input_ids']))

In [ ]:
test_dataset=tf.data.Dataset.from_tensor_slices((dict(xtest_encoding),ytest_encoding['input_ids']))

In [ ]:
train_dataset = train_dataset.batch(4)
test_dataset=test_dataset.batch(4)

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-hi")


All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
import datasets
tf_train_dataset = model.prepare_tf_dataset(
    train_dataset,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=4,
)
tf_eval_dataset = model.prepare_tf_dataset(
    test_dataset,
    collate_fn=data_collator,
    shuffle=False,
    batch_size=4,
)

ImportError: ignored

In [ ]:
! pip install datasets
! pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org
! pip install datasets sacrebleu torch transformers sentencepiece transformers[sentencepiece]
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
ERROR: You must give at least one requirement to install (see "pip help install")
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.9 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset
from datasets import load_metric


In [ ]:
import pandas as pd
df=pd.read_csv("/content/english-to_hindi_dataset.csv", encoding='utf-8')

df = df.dropna()

In [ ]:

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['input_text'],df['target_text'],test_size=0.2)
X_test,X_val,y_test,y_val=train_test_split(X_test,y_test,test_size=0.2)

In [ ]:

def prep_data_for_model_fine_tuning(source_lang: list, target_lang: list) -> list:
    """Takes the input data lists and converts into translation list of dicts"""

    data_dict = dict()
    data_dict['translation'] = []

    for sr_text, tr_text in zip(source_lang, target_lang):
        temp_dict = dict()
        temp_dict['en'] = sr_text
        temp_dict['hi'] = tr_text

        data_dict['translation'].append(temp_dict)

    return data_dict

In [ ]:
training_data = prep_data_for_model_fine_tuning(X_train.values, y_train.values)
validation_data = prep_data_for_model_fine_tuning(X_val.values, y_val.values)
test_data = prep_data_for_model_fine_tuning(X_test.values, y_test.values)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
def generate_model_ready_dataset(dataset: list, source: str, target: str,
                                 model_checkpoint: str,
                                 tokenizer: AutoTokenizer):
    """Makes the data training ready for the model"""

    preped_data = []

    for row in dataset:
        inputs = " " + row[source]
        targets = row[target]

        model_inputs = tokenizer(inputs, max_length=1414,
                                 truncation=True, padding=True)

        model_inputs['translation'] = row

        # setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=1414,
                                 truncation=True, padding=True)
            model_inputs['labels'] = labels["input_ids"]

        preped_data.append(model_inputs)

    return preped_data


In [ ]:

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

train_data = generate_model_ready_dataset(dataset=training_data['translation'],
                                          tokenizer=tokenizer,
                                          source='en',
                                          target='hi',
                                          model_checkpoint='"Helsinki-NLP/opus-mt-en-hi"')
validation_data = generate_model_ready_dataset(dataset=validation_data['translation'],
                                               tokenizer=tokenizer,
                                               source='en',
                                               target='hi',
                                               model_checkpoint='"Helsinki-NLP/opus-mt-en-hi"')
test_data = generate_model_ready_dataset(dataset=test_data['translation'],
                                         tokenizer=tokenizer,
                                         source='en',
                                         target='hi',
                                         model_checkpoint='"Helsinki-NLP/opus-mt-en-hi"')

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
# Convert to DataFrame
train_df = pd.DataFrame.from_records(train_data)
validation_df = pd.DataFrame.from_records(validation_data)
test_df = pd.DataFrame.from_records(test_data)

# Convert DataFrames to Dataset objects
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:

model_args = Seq2SeqTrainingArguments(
    output_dir="/content/trained/fine-tuned-en-hi",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.02,
    save_total_limit=3,
    num_train_epochs=30,
    predict_with_generate=True
)

In [ ]:
BLEU='bleu'
import numpy as np
from datasets import load_metric
def postprocess_text(preds: list, labels: list) -> tuple:

    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds: tuple) -> dict:

    metric = load_metric("sacrebleu")
    tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {BLEU: result['score']}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]

    result['gen_len'] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
trainer = Seq2SeqTrainer(
    model,
    model_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,a
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,2.267421,26.953300,28.462300
2,2.958600,2.395660,20.790800,20.665800
3,2.082900,2.610351,22.634000,18.346700
4,1.489200,2.829392,22.988900,18.128100
5,1.042000,2.962271,23.540900,17.693500
6,0.710600,3.136864,24.666800,18.010100
7,0.501300,3.258025,26.274300,15.839200
8,0.371800,3.379570,25.986400,16.711100
9,0.288000,3.457696,21.678400,15.412100
10,0.234500,3.535841,24.678000,16.733700


<ipython-input-24-62294b954b6b>:13: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: Fut

TrainOutput(global_step=14910, training_loss=0.3690917041140223, metrics={'train_runtime': 3996.2975, 'train_samples_per_second': 59.673, 'train_steps_per_second': 3.731, 'total_flos': 3812302016151552.0, 'train_loss': 0.3690917041140223, 'epoch': 30.0})

In [ ]:
trainer.save_model("/content/trained/fine-tuned-en-hi-Transformer")


In [ ]:
model.save_pretrained("/content/fine-tuned-en-hi-Transformer-model")

tokenizer.save_pretrained("/content/fine-tuned-en-hi-Transformer-tokenizer")

AttributeError: ignored

In [ ]:
!zip -r /content/fine-tuned-en-hi-Transformer.zip /content/fine-tuned-en-hi-Transformer
from google.colab import files
#files.download("/content/trained.zip")

  adding: content/fine-tuned-en-hi-Transformer/ (stored 0%)
  adding: content/fine-tuned-en-hi-Transformer/special_tokens_map.json (deflated 35%)
  adding: content/fine-tuned-en-hi-Transformer/generation_config.json (deflated 43%)
  adding: content/fine-tuned-en-hi-Transformer/model.safetensors (deflated 7%)
  adding: content/fine-tuned-en-hi-Transformer/target.spm (deflated 60%)
  adding: content/fine-tuned-en-hi-Transformer/config.json (deflated 61%)
  adding: content/fine-tuned-en-hi-Transformer/vocab.json (deflated 76%)
  adding: content/fine-tuned-en-hi-Transformer/tokenizer_config.json (deflated 68%)
  adding: content/fine-tuned-en-hi-Transformer/training_args.bin (deflated 50%)
  adding: content/fine-tuned-en-hi-Transformer/source.spm (deflated 51%)


In [ ]:
test_results = trainer.predict(test_dataset)
print("Test Bleu Score: ", test_results.metrics["test_bleu"])

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Test Bleu Score:  14.5189


In [ ]:
import torch
from tqdm import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

predictions = []
test_input = test_dataset['translation']

for input_text in tqdm(test_input):
    source_sentence = input_text['en']
    encoded_source = tokenizer(source_sentence,
                               return_tensors='pt',
                               padding=True,
                               truncation=True)
    encoded_source.to(device)  # Move input tensor to the same device as the model

    translated = model.generate(**encoded_source)

    predictions.append([tokenizer.decode(t, skip_special_tokens=True) for t in translated][0])

# Move the model back to CPU if needed
model.to("cpu")


100%|██████████| 1590/1590 [04:37<00:00,  5.73it/s]


MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(61950, 512, padding_idx=61949)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(61950, 512, padding_idx=61949)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,),

In [ ]:
y_true_en = []
y_true_pt = []
from tqdm import tqdm

for input_text in tqdm(test_input):
    y_true_pt.append(input_text['en'])
    y_true_en.append(input_text['hi'])
output_df = pd.DataFrame({"y_true_en": y_true_pt, "y_true_hi": y_true_en, "predicted_text": predictions})
output_df

100%|██████████| 1590/1590 [00:00<00:00, 712874.76it/s]


,y_true_en,y_true_hi,predicted_text
0,They seek to extinguish the Light of Allah wit...,वे चाहते है कि अल्लाह के प्रकाश को अपने मुँह क...,वे चाहते है कि अल्लाह के प्रकाश को अपने मुँह क...
1,Deselect All,सीआरएल फ़ाइल चुनें,सभी अचयनित करें (l)
2,"Say (O Muhammad SAW): ""Call unto those besides...","कह दो, ""तुम उससे इतर जिनको भी पूज्य-प्रभु समझत...","कह दो, ""तुम उससे इतर जिनको भी पूज्य-प्रभु समझत..."
3,"""And sit not on every road, threatening, and h...","""और प्रत्येक मार्ग पर इसलिए न बैठो कि धमकियाँ ...","""और प्रत्येक मार्ग पर इसलिए न बैठो कि धमकियाँ ..."
4,"Thus, there exist several opportunities to har...",इस प्रकार से राज्य में सन्निहित इन प्राकृतिक ल...,"अतः, सरकार में इन प्राकृतिक लाभों को चलाने के ..."
...,...,...,...
1585,This is a list of e - mail addresses or domain...,यह स्पैम फिल्टर में संचित ई-मेल पता या डोमेन न...,यह स्पैम स्पैम सर्वरों में ईमेल पता या डोमेन न...
1586,Pacific / Port _ Moresby,प्रशांत/पॉर्ट _ मोरेस्बी,प्रशांत/ पोर्ट मोरेसबी
1587,carelessness,अनाचिती,लापरवाही
1588,Industrial fluctuation may be a seasonal trend...,औद्योगिक उतार-चढाव एक मौसमी प्रवृत्ति भी हो सक...,यह एक ऐसा चलन है जो आज दुनिया में आम तौर पर दे...


In [ ]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
model_path = '/content/fine-tuned-en-hi-Transformer'


model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

tokenizer = AutoTokenizer.from_pretrained(model_path)

input_text = input("enter your text :")
input_ids = tokenizer(input_text, return_tensors='pt').input_ids
output = model.generate(input_ids)
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Answer:", decoded_output)

enter your text :how are you boy. Lets meet tomorrow
Generated Answer: आप अपने लड़के से कैसे मिल रहे है। चलो कल


HINGLISH TO ENGLISH

In [1]:
! pip install datasets
! pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org
! pip install datasets sacrebleu torch transformers sentencepiece transformers[sentencepiece]
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
ERROR: You must give at least one requirement to install (see "pip help install")
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from datasets import load_dataset

dataset = load_dataset("findnitai/english-to-hinglish")



Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
import pandas as pd
df=pd.DataFrame(dataset['train']['translation'])
df.drop(['source'],axis=1)
df.head()


,en,hi_ng,source
0,What's the name of the movie,film ka kya naam hai,1
1,"Hi, the rotten tomatoes score is great but the...","namaste, sada hua tomatoes score mahaan hai, l...",1
2,Do you think you will like the movie,kya aapako lagata hai ki aapako film pasand aa...,1
3,What kind of movie is it,yah kis tarah kee philm hai,1
4,when was the movie made?,film kab banee thee?,1


In [4]:
from datasets import Dataset
from datasets import load_metric

df1 = df.sample(n=15000, random_state=42)


In [5]:

df1 = df1.dropna()

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df1['en'],df1['hi_ng'],test_size=0.2)
X_test,X_val,y_test,y_val=train_test_split(X_test,y_test,test_size=0.2)

In [6]:

def prep_data_for_model_fine_tuning(source_lang: list, target_lang: list) -> list:
    """Takes the input data lists and converts into translation list of dicts"""

    data_dict = dict()
    data_dict['translation'] = []

    for sr_text, tr_text in zip(source_lang, target_lang):
        temp_dict = dict()
        temp_dict['en'] = sr_text
        temp_dict['hi_ng'] = tr_text

        data_dict['translation'].append(temp_dict)

    return data_dict

In [7]:
training_data = prep_data_for_model_fine_tuning(X_train.values, y_train.values)
validation_data = prep_data_for_model_fine_tuning(X_val.values, y_val.values)
test_data = prep_data_for_model_fine_tuning(X_test.values, y_test.values)

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
def generate_model_ready_dataset(dataset: list, source: str, target: str,
                                 model_checkpoint: str,
                                 tokenizer: AutoTokenizer):
    """Makes the data training ready for the model"""

    preped_data = []

    for row in dataset:
        inputs = " " + row[source]
        targets = row[target]

        model_inputs = tokenizer(inputs, max_length=1414,
                                 truncation=True, padding=True)

        model_inputs['translation'] = row

        # setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=1414,
                                 truncation=True, padding=True)
            model_inputs['labels'] = labels["input_ids"]

        preped_data.append(model_inputs)

    return preped_data


In [9]:

# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("findnitai/t5-hinglish-translator")
model = AutoModelForSeq2SeqLM.from_pretrained("findnitai/t5-hinglish-translator")
train_data = generate_model_ready_dataset(dataset=training_data['translation'],
                                          tokenizer=tokenizer,
                                          source='en',
                                          target='hi_ng',
                                          model_checkpoint='"findnitai/t5-hinglish-translator"')
validation_data = generate_model_ready_dataset(dataset=validation_data['translation'],
                                               tokenizer=tokenizer,
                                               source='en',
                                               target='hi_ng',
                                               model_checkpoint='"findnitai/t5-hinglish-translator"')
test_data = generate_model_ready_dataset(dataset=test_data['translation'],
                                         tokenizer=tokenizer,
                                         source='en',
                                         target='hi_ng',
                                         model_checkpoint='"findnitai/t5-hinglish-translator"')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [10]:
# Convert to DataFrame
train_df = pd.DataFrame.from_records(train_data)
validation_df = pd.DataFrame.from_records(validation_data)
test_df = pd.DataFrame.from_records(test_data)

# Convert DataFrames to Dataset objects
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)

In [11]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [12]:
model_args = Seq2SeqTrainingArguments(
    output_dir="/content/FineTune-en-hi_ng",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.02,
    save_total_limit=3,
    num_train_epochs=30,
    predict_with_generate=True
)

In [13]:
BLEU='bleu'
import numpy as np
from datasets import load_metric
def postprocess_text(preds: list, labels: list) -> tuple:

    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds: tuple) -> dict:

    metric = load_metric("sacrebleu")
    tokenizer = AutoTokenizer.from_pretrained("findnitai/t5-hinglish-translator")

    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {BLEU: result['score']}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]

    result['gen_len'] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result

In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

trainer = Seq2SeqTrainer(
    model,
    model_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [15]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,2.591400,2.444203,30.639200,11.080000
2,2.341000,2.265941,31.919800,10.825000
3,2.114400,2.161456,34.448600,10.980000
4,1.947800,2.048343,36.082200,10.998300
5,1.792000,1.999925,37.161700,11.161700
6,1.720400,1.964107,38.650200,11.213300
7,1.596600,1.914286,39.059000,11.293300
8,1.475800,1.901515,39.166400,11.248300
9,1.391800,1.896929,40.139500,11.383300
10,1.313700,1.869694,40.086900,11.290000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
<ipython-input-13-60553df73239>:13: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mand

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We reco

TrainOutput(global_step=45000, training_loss=1.2708735087076823, metrics={'train_runtime': 4230.277, 'train_samples_per_second': 85.101, 'train_steps_per_second': 10.638, 'total_flos': 1997183981715456.0, 'train_loss': 1.2708735087076823, 'epoch': 30.0})

In [16]:
trainer.save_model("/content/fine-tuned-en-hi_ng-Transformer")

In [ ]:
test_results = trainer.predict(test_dataset)
print("Test Bleu Score: ", test_results.metrics["test_bleu"])

In [22]:
import locale

# Set the locale to a UTF-8 locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')




'en_US.UTF-8'

In [26]:
import shutil
import os

folder_to_zip = 'fine-tuned-en-hi_ng-Transformer'

shutil.make_archive('/content/fine-tuned-en-hi_ng-Transformer', 'zip', folder_to_zip)

shutil.move(f'{folder_to_zip}.zip', '/content')


download_link = f'/content/{folder_to_zip}.zip'

print(f'Download your zip file here: {download_link}')

Error: ignored

In [24]:
!zip -r /content/fine-tuned-en-hi_ng-Transformer.zip  /content/fine-tuned-en-hi_ng-Transformer
#from google.colab import files
#files.download("/content/trained.zip")

NotImplementedError: ignored

In [19]:
import torch
from tqdm import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

predictions = []
test_input = test_dataset['translation']

for input_text in tqdm(test_input):
    source_sentence = input_text['en']
    encoded_source = tokenizer(source_sentence,
                               return_tensors='pt',
                               padding=True,
                               truncation=True)
    encoded_source.to(device)  # Move input tensor to the same device as the model

    translated = model.generate(**encoded_source)

    predictions.append([tokenizer.decode(t, skip_special_tokens=True) for t in translated][0])

# Move the model back to CPU if needed
model.to("cpu")


  0%|          | 0/2400 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 2400/2400 [04:43<00:00,  8.47it/s]


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [20]:
y_true_en = []
y_true_pt = []
from tqdm import tqdm

for input_text in tqdm(test_input):
    y_true_pt.append(input_text['en'])
    y_true_en.append(input_text['hi_ng'])
output_df = pd.DataFrame({"y_true_en": y_true_pt, "y_true_hi_ng": y_true_en, "predicted_text": predictions})
output_df

100%|██████████| 2400/2400 [00:00<00:00, 782215.37it/s]


,y_true_en,y_true_hi_ng,predicted_text
0,remind me of dog ' s vet appointment on the 17...,Muje dog ki vet appointment ke liye 17 ko do d...,mujhe 17 do din pehle dog ke vet appointment k...
1,Re - alert me via alarm in 10 minutes .,10 minutes me mujhe alarm ke zariye yaad dilaa...,10 minutes me alarm ko alert karen.
2,Please show me a route with less traffic,Please mujhe sabse kam traffic wala raasta dikhao,Please mujhe wo raasta ka traffic bataye
3,create an alarm for Monday at 8,Monday 8 baje ke liye ek alarm banaen,Monday 8 baje ke liye alarm create kare
4,current traffic to beach,beach tak current traffic,beach par current traffic
...,...,...,...
2395,I don ' t need that alarm anymore .,mujhe ab us alarm ki zarurat nae hai,mujhe wo alarm chahiye.
2396,Set up alarm for 10 PM .,10 PM ke liye alarm set karen .,10 PM ke liye alarm set kare
2397,Set a reminder for five tonight to turn off th...,Aaj raat five baje ke liye ek reminder set kar...,Aaj raat five ke liye crockpot band karne ke l...
2398,Set daily alarms for 6 and 7 pm,6 aur 7 pm ke liye daily alarms set karen,6 aur 7 pm ke liye daily alarms set karen


In [28]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
model_path = '/content/fine-tuned-en-hi_ng-Transformer'


model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

tokenizer = AutoTokenizer.from_pretrained(model_path)

input_text = input("enter your text :")
input_ids = tokenizer(input_text, return_tensors='pt').input_ids
output = model.generate(input_ids)
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Answer:", decoded_output)

enter your text :where should I go
Generated Answer: mujhe kahan jaana chahiye
